# Data

# Preprocessing

In [1]:
from datasets import load_dataset

basic_dataset = load_dataset("json", data_files="./Data/wholeqa.jsonl", split='train')

Found cached dataset json (/home/yunho/.cache/huggingface/datasets/json/default-7673f6a0c673c641/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [2]:
from datasets import DatasetDict

split_dataset = basic_dataset.train_test_split(test_size=0.1)

train_dataset = split_dataset['train']
valid_dataset = split_dataset['test']

wholeqa_dataset = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset
})

print(wholeqa_dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 12664
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1408
    })
})


## Making input

In [3]:
qa_text = """다음과 같은 법률 상담 질문이 있다.

{}

이 사건에 대한 법률 상담 답변은 다음과 같다.

"""

In [4]:
print(type(wholeqa_dataset))


<class 'datasets.dataset_dict.DatasetDict'>


In [5]:
import numpy as np

idx = np.random.randint(0, len(wholeqa_dataset['train']))

question = wholeqa_dataset['train'][idx]['instruction']
answer = wholeqa_dataset['train'][idx]['output']

print(qa_text.format(question))
print(answer)

다음과 같은 법률 상담 질문이 있다.

이행지체 중의 양도소득세 부과분에 대한 매수인의 책임여부: 甲은 A토지에 관하여 乙과 매매계약을 체결하였는데, 乙은 甲과 약정한 잔금 지급기일에 잔금을 지급하지 않았습니다(하지만 甲은 乙의 잔금지급 지체를 이유로 매매계약을 해제하지는 않았습니다). 그런데 乙이 잔금지급을 지체하고 있는 사이에 A토지의 개별공시지가는 급등하였고, 이로 인하여 甲은 추가로 양도소득세를 부담하게 되었습니다. 이에 甲은 乙로부터 乙의 잔금지급 지체로 인하여 추가로 부담하게 된 양도소득세의 차액부분을 지급받을 수 있는지요?

이 사건에 대한 법률 상담 답변은 다음과 같다.


(1)「민법」은 채무불이행으로 인한 손해배상은 통상의 손해를 그 한도로 하고, 특별한 사정으로 인한 손해는 채무자가 그 사정을 알았거나 알 수 있었을 때에 한하여 배상의 책임이 있다고 규정하고 있습니다(같은 법 제393조).
  (2) 통상손해란 채무불이행이 있으면 일반적으로 발생하는 것으로 여겨지는 손해를 말하고, 특별손해란 채무불이행으로 인해 일반적으로 발생하는 손해가 아닌 채권자에게 존재하는 특별한 사정에 기초하여 발생하는 손해를 뜻합니다. 따라서 채무자는 채무불이행으로 인하여 채권자가 입게 된 통상손해에 대해서는 그 전부에 대하여 배상책임을 부담하게 되지만, 특별손해에 대해서는 채권자에게 존재하는 특별한 사정에 관하여 채무자가 ‘알았거나 알 수 있었을 때’에 한하여 예외적으로 배상책임을 부담하게 됩니다.
  (3) 위 사안에서 판례는 “매수인의 잔금지급 지체로 인하여 계약을 해제하지 아니한 매도인이 지체된 기간 동안 입은 손해 중 그 미지급 잔금에 대한 법정이율에 따른 이자 상당의 금액은 통상손해라고 할 것이지만, 그 사이에 매매대상 토지의 개별공시지가가 급등하여 매도인의 양도소득세 부담이 늘었다고 하더라도 그 손해는 사회일반의 관념상 매매계약에서의 잔금지급의 이행지체의 경우 통상 발생하는 것으로 생각되는 범위의 통상손해라고 할 수는 없고, 이는 특별한 사정에 의하여 

In [6]:
qa_dataset = wholeqa_dataset.map(lambda x: {
    'text': qa_text.format(x['instruction'], x['output'])
})

Map:   0%|          | 0/12664 [00:00<?, ? examples/s]

Map:   0%|          | 0/1408 [00:00<?, ? examples/s]

In [7]:
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 12664
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 1408
    })
})

In [8]:
import torch as th
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_ckpt = "beomi/llama-2-ko-7b"
# model_ckpt = "kfkas/Llama-2-ko-7b-Chat"

bnb_config = BitsAndBytesConfig(
    load_in_8bits=True, # 8bit quantization
    bnb_8bit_use_double_quant=True, # 양자화 상수 중복 계산
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=th.bfloat16 
)

In [9]:
# tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, padding_side='right')
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer.eos_token, tokenizer.eos_token_id)

</s> 2


In [10]:
model = AutoModelForCausalLM.from_pretrained(model_ckpt, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [11]:
def encode(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
    labels = tokenizer(examples['output'], truncation=True, padding='max_length', max_length=512)
    inputs['labels'] = labels['input_ids']
    return inputs

qa_dataset = qa_dataset.map(encode, batched=True)

Map:   0%|          | 0/12664 [00:00<?, ? examples/s]

Map:   0%|          | 0/1408 [00:00<?, ? examples/s]

In [12]:
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 12664
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1408
    })
})

In [13]:
print(tokenizer.decode(qa_dataset['train']['input_ids'][0]))
print('=' * 100)
print(tokenizer.decode(qa_dataset['train']['labels'][0]))

2023-08-31 08:16:19.999672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 08:16:20.111766: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-31 08:16:20.618732: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/mesa-diverted/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/mesa:/usr/lib/x86_64-

<s> 다음과 같은 법률 상담 질문이 있다.

甲은 근로복지공단으로부터 폐질 3급 4호에 해당한다는 이유로 상병보상연금을 지급받고 있었습니다. 그런데, A의 불법행위로 인해 甲이 사망하는 사고가 발생하였습니다.A의 손해배상금을 산정함에 있어 甲이 상병보상연금을 지급받고 있었다는 사실이 일실수익의 산정 또는 과실상계에 반영될 여지가 있는지요?

이 사건에 대한 법률 상담 답변은 다음과 같다.

</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s

In [14]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [15]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [16]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8, # attention dim
    lora_alpha=32, # LoRA 스케일링 파라미터
    target_modules=["q_proj", "v_proj"], # LoRA 적용할 모듈
    # target_modules=["query_key_value"] : 오류 발생(이유 모름)
    lora_dropout=0.05, # LoRA 드롭아웃
    bias="none",
    task_type="CAUSAL_LM",
)

peft_config.inference_mode = False
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,860,050,432 || trainable%: 0.06114100824149743


In [17]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', th.cuda.current_device())
print('Count of using GPUs:', th.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 4


In [18]:
qa_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
qa_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 12664
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1408
    })
})

In [19]:
import transformers

args = transformers.TrainingArguments(
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    warmup_steps=200,
    num_train_epochs=2,
    learning_rate=2e-4,
    # max_grad_norm=2.0,
    fp16=True,
    logging_steps=10,
    logging_strategy='steps',
    output_dir="outputs",
    optim="paged_adamw_8bit",
    overwrite_output_dir=True,
    save_strategy="epoch",
    save_steps=500,
    save_total_limit=1,
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    report_to = 'tensorboard'
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=qa_dataset['train'],
    eval_dataset=qa_dataset['validation'],
    args=args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

In [20]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("/home/yunho/legal_basic/llama2_legal_basic")
tokenizer.save_pretrained("/home/yunho/legal_basic/llama2_legal_basic")